In [1]:
import torch
import torch.nn as nn
import torch.onnx
import torchvision

MODEL_PATH = "/home/namdng/garbage_classifier/models/lr_1e-3_bs_64_sche-f0.2-p6/ckpt_63_0.9641_.pth"
IMAGE_SIZE = 394
BATCH_SIZE = 1

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Load model
class_names = ["cardboard_paper", "glass", "metal", "others", "plastic"]
model_conv = torchvision.models.efficientnet_v2_s()
num_ftrs = model_conv.classifier[1].in_features
model_conv.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(num_ftrs, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, len(class_names))
)
model_conv = model_conv.to(device)
model_conv.load_state_dict(torch.load(MODEL_PATH, weights_only=True))
model_conv.eval()

# Input to the model
x = torch.randn(BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE, device=device)
torch_out = model_conv(x)
print(torch_out)



tensor([[ -0.9138,  -1.5270,  -6.5463,  -3.5999, -11.1261]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


### TorchScript-based ONNX Exporter

In [ ]:
# Export the model
torch.onnx.export(model_conv,                # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "gc_torchscript.onnx",                 # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

### TorchDynamo-based ONNX Exporter

In [2]:
onnx_program = torch.onnx.export(model_conv, x, dynamo=True)
onnx_program.save("gc_torchdynamo.onnx")

/data/anaconda3/envs/vision/lib/python3.11/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/data/anaconda3/envs/vision/lib/python3.11/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


[torch.onnx] Obtain model graph for `EfficientNet([...]` with `torch.export.export`...
[torch.onnx] Obtain model graph for `EfficientNet([...]` with `torch.export.export`... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


In [3]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession("gc_torchdynamo.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

print(to_numpy(torch_out), ort_outs[0])

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

[[ -0.91379267  -1.5269971   -6.5462546   -3.5999322  -11.126145  ]] [[ -0.91379535  -1.5270008   -6.5462513   -3.599944   -11.126142  ]]
Exported model has been tested with ONNXRuntime, and the result looks good!


2024-11-05 17:38:07.480124503 [W:onnxruntime:, graph.cc:4198 CleanUnusedInitializersAndNodeArgs] Removing initializer 'val_429'. It is not used by any node and should be removed from the model.
2024-11-05 17:38:07.480139909 [W:onnxruntime:, graph.cc:4198 CleanUnusedInitializersAndNodeArgs] Removing initializer 'val_417'. It is not used by any node and should be removed from the model.
2024-11-05 17:38:07.480142891 [W:onnxruntime:, graph.cc:4198 CleanUnusedInitializersAndNodeArgs] Removing initializer 'val_414'. It is not used by any node and should be removed from the model.
2024-11-05 17:38:07.480145257 [W:onnxruntime:, graph.cc:4198 CleanUnusedInitializersAndNodeArgs] Removing initializer 'val_405'. It is not used by any node and should be removed from the model.
2024-11-05 17:38:07.480147518 [W:onnxruntime:, graph.cc:4198 CleanUnusedInitializersAndNodeArgs] Removing initializer 'val_397'. It is not used by any node and should be removed from the model.
2024-11-05 17:38:07.480149768 

### convert test images to RGB

In [ ]:
import os
from PIL import Image

DATA_DIR = "data_split/val"

for class_name in os.listdir(DATA_DIR):
    class_dir = os.path.join(DATA_DIR, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = Image.open(image_path)
        image_mode = image.mode
        if image_mode != "RGB":
            image = image.convert("RGB")
            image.save(image_path)